In [1]:
from src.circuit import Adder, AdderRev, ModAdder, ModAdderRev
from src.circuit import ModMulti7xmod15, ModExp
from src.utils import c2q, q2c

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


# 加法器、减法器

无符号4-bit加法器和减法器线路如下图所示。a3-a2-a1-a0 和 b3-b2-b1-b0 分别记作 a 和 b，则加法器计算 b + a， 减法器（逆加法器）计算 b - a. 最终的计算结果保存在 b 中。

<img src="./images/Fig6_adder.jpg" width=400>

<img src="./images/Fig7_subtractor.jpg" width=400>

In [2]:
def demo1():
    """验证 Fig.6（加法器）, Fig.7 （减法器）
    """
    print("验证加法器,减法器:")
    # 验证 b + a, b - a
    a = 3
    b = 8
    cir_a = c2q([3, 2, 1, 0], a)
    cir_b = c2q([7, 6, 5, 4], b)

    cir_add = cir_a + cir_b + Adder()
    q_add = q2c(cir_add, [7, 6, 5, 4])

    cir_sub = cir_a + cir_b + AdderRev()
    q_sub = q2c(cir_sub, [7, 6, 5, 4])

    print(
        f"q_add = {q_add}, c_add = {b + a}\nq_sub = {q_sub}, c_sub = {b - a}\n")

demo1()

验证加法器,减法器:
ket string: 1¦0000010110011⟩
ket string: 1¦0010001010011⟩
q_add = 11, c_add = 11
q_sub = 5, c_sub = 5



打印的结果中， `q_` 开头代表量子计算的结果，`c_` 开头代表经典计算机或正确的结果，`add` 和 `sub` 分别代表加法器和减法器，`ket string` 是线路的终态。以 `a = 3, b = 8` 为例，量子线路结果正确。

# 模加法器、模减法器

与加法器和减法器相比，引入模运算后会将加减的结果求模，结果保存在寄存器b中。Fig.8 和 Fig.9 中的最左和最右竖直灰色线条分割开输入和输出，中间是运算器主体部分。后文类似，不再赘述。

<img src="./images/Fig8_modular_adder.jpg" width=400>

<img src="./images/Fig9_rev_modular_adder.jpg" width=400>

In [7]:
def demo2():
    """验证模加法器、模减法器
    """
    print("验证模加法器、模减法器:")
    # 1. 计算 (a + b) mod n
    # 2. 计算 (b - a) mod n。注：对 b<a, (b-a) mod n = (b-a+n) mod n
    a = 11
    b = 9
    n = 15
    # 将经典数值转换成量子线路编码
    cir_a = c2q([3, 2, 1, 0], a)
    cir_b = c2q([7, 6, 5, 4], b)
    cir_n = c2q([16, 15, 14, 13], n)  # 输入置1

    cir_madd = cir_a + cir_b + cir_n + ModAdder() + cir_n
    cir_msub = cir_a + cir_b + cir_n + ModAdderRev() + cir_n
    # 使用量子线路计算结果
    q_add = q2c(cir_madd, [7, 6, 5, 4])
    q_sub = q2c(cir_msub, [7, 6, 5, 4])
    # 经典计算验证
    c_add = (a + b) % n
    c_sub = (b - a) % n

    print(
        f"q_add = {q_add}, c_add = {c_add}\nq_sub = {q_sub}, c_sub = {c_sub}\n")

demo2()

验证模加法器、模减法器:
ket string: 1¦000000000001011011⟩
ket string: 1¦000000000011011011⟩
q_add = 5, c_add = 5
q_sub = 13, c_sub = 13



# 模乘法器

以 $7 x\mod 15$ 为例，其线路如 Fig.10 所示。在 8U32G 硬件平台上，运行模乘法器耗时大约 20 秒。

<img src="./images/Fig10_7xmod15.jpg" width=400>

In [8]:
def demo3():
    """验证模乘法器
    """
    print("验证模乘法器(8U32G硬件大约耗时20秒):")
    # 计算 7x mod n
    ctrl = 1  # 控制位
    x = 9
    n = 15
    cir_ctrl = c2q([0], ctrl)
    cir_x = c2q([4, 3, 2, 1], x)
    cir_n = c2q([21, 20, 19, 18], n)
    cir_7x = cir_ctrl + cir_x + cir_n + ModMulti7xmod15() + cir_n
    # 获取终态得到结果
    q_mul = q2c(cir_7x, [12, 11, 10, 9])
    c_mul = (7 * x) % 15

    print(f"q_mul = {q_mul}, c_mul = {c_mul}\n")
    
demo3()

验证模乘法器(8U32G硬件大约耗时20秒):
ket string: 1¦00000000000011000010011⟩
q_mul = 3, c_mul = 3



# 模指数器

以 $7^3 \mod 15$ 为例， 其线路如图 Fig.12 所示。在 8U32G 硬件平台上，运行模乘法器耗时大约 20 分钟。最终的结果存储在 `regx` 中。从运行结果可以看出，量子线路的计算结果正确。

<img src="./images/Fig12_modular_exp.jpg" width=600>

In [3]:
def demo4():
    """验证模指数器
    """
    # 计算 7^a mod n
    print("验证模指数器(8U32G硬件大约耗时20分钟):")
    a = 3
    n = 15
    cir_a = c2q([3, 2, 1, 0], a)
    cir_n = c2q([25, 24, 23, 22], n)
    cir_expmod = cir_a + cir_n + ModExp() + cir_n

    q_res = q2c(cir_expmod, [8, 7, 6, 5])
    c_res = (7**a) % n

    print(f"q_res = {q_res}, c_res = {c_res}\n")

demo4()

验证模指数器(8U32G硬件大约耗时20分钟):
ket string: 1¦000000000000000000110100011⟩
q_res = 13, c_res = 13

